<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/104_Customization_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#104自訂條件職缺爬蟲(暫時版)

In [69]:
ID = "*********" #輸入帳號
password = "*********" #輸入密碼
job_op = '#li_newjob1'



In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
#!apt-get install xvfb
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import WebDriverWait

!pip install fake_useragent
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import time

from google.colab import drive
drive.mount('/content/drive')

In [64]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [65]:
url = "https://www.104.com.tw/jobs/main/"
browser.get(url)
time.sleep(3) #暫停

#進入登入頁面
signin = browser.find_element_by_css_selector('#global_bk > ul > li.right > ul > li:nth-child(6) > a')
browser.execute_script("arguments[0].click();", signin)
time.sleep(1) #暫停

#填帳號密碼與登入
context = browser.find_element(by=By.XPATH, value='//*[@id="username"]')
context.send_keys(ID)
time.sleep(0.2) #暫停
context = browser.find_element(by=By.XPATH, value='//*[@id="password"]')
context.send_keys(password)
time.sleep(0.2) #暫停

submit = browser.find_element(by=By.XPATH, value='//*[@id="submitBtn"]/h2')
submit.click() #點擊登入
time.sleep(5) #暫停


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [70]:
#顯示自訂工作列表
newjob = browser.find_element_by_css_selector('#bubjobs_li > a')
browser.execute_script("arguments[0].click();", newjob) #點擊
time.sleep(5) #暫停

#自訂工作
newjob = browser.find_element_by_css_selector(job_op)
browser.execute_script("arguments[0].click();", newjob)

#檢查並切換頁面
n = browser.window_handles  # 生成一個新視窗或新頁籤的識別碼，代表這個窗口的模擬driver
print('當前識別碼:', n)  # 列印所有頁面識別碼
browser.switch_to.window(browser.window_handles[-1])  # 瀏覽器切换至最新跳出的頁面
time.sleep(5) #暫停

#擷取總頁數
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
last_page = soup.find('select', class_="form-control p-0 h3 h-auto font-weight-bold").find('option').text
last_page = last_page.split(' ')[-3]
last_page = int(last_page)+1

for i in range(last_page):
  browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
  t = random.uniform(3, 4) #隨機數
  time.sleep(t) #暫停

#擷取職缺網址
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
job_links = []
urls = soup.find_all('a', class_="info-job__text jb-link jb-link-blue jb-link-blue--visited h2")
for url in urls:
  link = url.get("href").replace('//', '')
  job_links.append(link)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  import sys


當前識別碼: ['CDwindow-9811C06EE25C6A0965FBEFD760BFB0A3', 'CDwindow-9A22722FBD4A8CFCECE2E260C5658CBC', 'CDwindow-4AAD4D36F2AE875BA722C0F393D19551']


In [72]:
job_links


['www.104.com.tw/job/6wlz1?jobsource=m104_apply',
 'www.104.com.tw/job/7bwnb?jobsource=m104_apply',
 'www.104.com.tw/job/7ddta?jobsource=m104_apply',
 'www.104.com.tw/job/7asdk?jobsource=m104_apply',
 'www.104.com.tw/job/6x0q4?jobsource=m104_apply',
 'www.104.com.tw/job/6wohk?jobsource=m104_apply',
 'www.104.com.tw/job/7jf9y?jobsource=m104_apply',
 'www.104.com.tw/job/6wnct?jobsource=m104_apply',
 'www.104.com.tw/job/78w57?jobsource=m104_apply',
 'www.104.com.tw/job/7iy0k?jobsource=m104_apply',
 'www.104.com.tw/job/48rwl?jobsource=m104_apply',
 'www.104.com.tw/job/6uwns?jobsource=m104_apply',
 'www.104.com.tw/job/7lfq5?jobsource=m104_apply',
 'www.104.com.tw/job/4x6os?jobsource=m104_apply',
 'www.104.com.tw/job/61735?jobsource=m104_apply',
 'www.104.com.tw/job/7j4q6?jobsource=m104_apply',
 'www.104.com.tw/job/6dmla?jobsource=m104_apply',
 'www.104.com.tw/job/7ktmq?jobsource=m104_apply',
 'www.104.com.tw/job/7lsxt?jobsource=m104_apply',
 'www.104.com.tw/job/61va2?jobsource=m104_apply',
